In [ ]:
%pip install pandas
%pip install numpy
%pip install sqlite3
%pip install plotly
%pip install ipython-sql
%pip install matplotlib
%pip install seaborn

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
!wsl --exec wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/LargeData/m4_survey_data.sqlite

In [ ]:
conn = sqlite3.connect("m4_survey_data.sqlite")

In [ ]:
%load_ext sql
%sql sqlite:///m4_survey_data.sqlite

In [ ]:
%sql SELECT COUNT(*) FROM master

In [ ]:
data = pd.read_sql_query("SELECT * FROM master", conn)

In [ ]:
data.head()

In [ ]:
for column, number_of_missing_values in data.isna().sum().items():
    if number_of_missing_values > 0:
        print(f"Column {column} has {number_of_missing_values} missing values")

In [ ]:
data.replace({ 'Age' : np.nan }, data.Age.mean(), inplace=True)

In [ ]:
data.ConvertedComp.hist()

In [ ]:
data.Age.plot(kind = 'box')

In [ ]:
data.plot(kind = 'scatter', x = 'Age', y = 'WorkWeekHrs')

In [ ]:
px.scatter(data, x = 'WorkWeekHrs', y = 'CodeRevHrs', size='Age')

In [ ]:
pieplotdata = pd.read_sql_query("SELECT DatabaseDesireNextYear FROM DatabaseDesireNextYear", conn).value_counts().head(5)
ax = pieplotdata.plot(kind = 'pie', autopct='%1.1f%%', y = 'DatabaseDesireNextYear')
ax.legend(bbox_to_anchor=(1, 1.02), loc = 'best')

In [ ]:
barplotdata = data[(data.Age >= 30) & (data.index <= 35)][['Age', 'WorkWeekHrs', 'CodeRevHrs']].groupby(["Age"]).agg(['median'])
barplotdata.plot(kind = 'bar', stacked=True)

In [ ]:
lineplotdata = data[(data.Age >= 45) & (data.Age <= 60)][['Age', 'ConvertedComp']].groupby(["Age"]).agg(['median'])
lineplotdata.plot(kind = 'line', y = 'ConvertedComp')

In [ ]:
mainbranch_count = data[['MainBranch']].value_counts()
mainbranch_count.plot(kind = 'barh')

In [ ]:
pivot_table = pd.pivot_table(data, index='CareerSat', columns='JobSat', values='Respondent', aggfunc='count', fill_value=0)

# Plotting
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, cmap='Blues', fmt='g')
plt.title('Career Satisfaction vs. Job Satisfaction')
plt.xlabel('Job Satisfaction')
plt.ylabel('Career Satisfaction')
plt.show()

In [ ]:
pivot_table = pd.pivot_table(data, index='OpSys', columns='OpenSource', values='Respondent', aggfunc='count', fill_value=0)

# Plotting
pivot_table.plot(kind='bar')
plt.title('Open Source Adoption Across Operating Systems')
plt.xlabel('Operating System')
plt.ylabel('Number of Respondents')
plt.legend(title='Comparison with Closed Source')
plt.xticks(rotation=0)
plt.show()

In [ ]:
pivot_table = pd.pivot_table(data, index='Employment', columns='MgrWant', values='Respondent', aggfunc='count', fill_value=0)

# Plotting
pivot_table.plot(kind='bar', stacked=True)
plt.title('Career Aspirations by Employment Status')
plt.xlabel('Employment Status')
plt.ylabel('Number of Respondents')
plt.legend(title='Desire to be a Manager')
plt.xticks(rotation=0)
plt.show()

In [ ]:
data.to_excel('./data.xlsx', index=False)

In [ ]:
conn.close()